In [5]:
import argparse
import time
import torch
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import pdb
import dill as pickle
import argparse
from Models import get_model
from Beam import beam_search
from nltk.corpus import wordnet
from torch.autograd import Variable
import re


In [6]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('-load_weights', type=str, default='tiny_train')
parser.add_argument('-k', type=int, default=3)
parser.add_argument('-max_len', type=int, default=80)
parser.add_argument('-d_model', type=int, default=512)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-src_lang', type=str, default='it')
parser.add_argument('-trg_lang', type=str, default='en')
parser.add_argument('-heads', type=int, default=8)
parser.add_argument('-dropout', type=int, default=0.1)
parser.add_argument('-no_cuda', action='store_true')
parser.add_argument('-floyd', action='store_true')
#-load_weights clir_it_en -src_lang it -trg_lang en -k 2
#opt = parser.parse_args()
opt = parser.parse_args(args=[])
opt.device = 0 if opt.no_cuda is False else -1
print(opt)
assert opt.k > 0
assert opt.max_len > 10

Namespace(d_model=512, device=0, dropout=0.1, floyd=False, heads=8, k=3, load_weights='tiny_train/', max_len=80, n_layers=6, no_cuda=False, src_lang='it', trg_lang='en')


In [10]:
print(opt)
SRC, TRG = create_fields(opt)
model = get_model(opt, len(SRC.vocab), len(TRG.vocab))

Namespace(d_model=512, device=0, dropout=0.1, floyd=False, heads=8, k=3, load_weights='tiny_train/', max_len=80, n_layers=6, no_cuda=False, src_lang='it', trg_lang='en')
loading spacy tokenizers...
loading presaved fields...
error opening SRC.pkl and TRG.pkl field files, please ensure they are in tiny_train//


NameError: name 'quit' is not defined

In [5]:
def get_synonym(word, SRC):
    syns = wordnet.synsets(word)
    for s in syns:
        for l in s.lemmas():
            if SRC.vocab.stoi[l.name()] != 0:
                return SRC.vocab.stoi[l.name()]

    return 0


def multiple_replace(dict, text):
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

def preprocess_sentence(sentence, model, opt, SRC, TRG):
    indexed = []
    sentence = SRC.preprocess(sentence)
    for tok in sentence:
        if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
            indexed.append(SRC.vocab.stoi[tok])
        else:
            indexed.append(get_synonym(tok, SRC))
    sentence = Variable(torch.LongTensor([indexed]))

def translate_sentence(sentence, model, opt, SRC, TRG):
    model.eval()
    indexed = []
    sentence = SRC.preprocess(sentence)
    for tok in sentence:
        if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
            indexed.append(SRC.vocab.stoi[tok])
        else:
            indexed.append(get_synonym(tok, SRC))
    sentence = Variable(torch.LongTensor([indexed]))
    if opt.device == 0:
        sentence = sentence.cuda()

    sentence = beam_search(sentence, model, SRC, TRG, opt)

    return multiple_replace({' ?': '?', ' !': '!', ' .': '.', '\' ': '\'', ' ,': ','}, sentence)


def translate(opt, model, SRC, TRG):
    sentences = opt.text.lower().split('.')
    translated = []

    for sentence in sentences:
        translated.append(translate_sentence(sentence + '.', model, opt, SRC, TRG).capitalize())

    return (' '.join(translated))



In [6]:
print(type(SRC))

<class 'torchtext.data.field.Field'>


In [7]:
print(type(TRG))

<class 'torchtext.data.field.Field'>


In [20]:
 opt.text = "metodi insegnamento inglese lingua straniera quali metodi usano scuole stati uniti insegnare studenti madre lingua inglese"

In [21]:
phrase = translate(opt, model, SRC, TRG)

In [22]:
print(TRG.vocab.itos[4880])
print(TRG.vocab.itos[1283])
print(TRG.vocab.itos[1977])

architecture
structure
version


In [23]:
print(phrase)